## Model dimensionality
PyBaMM currently supports three types of model dimensions for the current collectors: 0D, 1D and 2D. These can be configured using the "dimensionality" model option.

In [1]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm
import os
import numpy as np
import matplotlib.pyplot as plt
os.chdir(pybamm.__path__[0]+'/..')

ERROR: Invalid requirement: '#'


Note: you may need to restart the kernel to use updated packages.


In [2]:
model_0D = pybamm.lithium_ion.SPMe({"dimensionality": 0})

In [3]:
param = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Marquis2019)
for p in param.keys():
    if 'heat transfer coefficient' in p:
        param.update({p:1.0})

In [4]:
param.search('heat transfer coefficient')

Edge heat transfer coefficient [W.m-2.K-1]	1.0
Negative current collector surface heat transfer coefficient [W.m-2.K-1]	1.0
Negative tab heat transfer coefficient [W.m-2.K-1]	1.0
Positive current collector surface heat transfer coefficient [W.m-2.K-1]	1.0
Positive tab heat transfer coefficient [W.m-2.K-1]	1.0
Total heat transfer coefficient [W.m-2.K-1]	1.0


In [5]:
sim_0D = pybamm.Simulation(model_0D, parameter_values=param)
sim_0D.solve([0, 3600])

In [6]:
sim_0D.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

The default dimensionality for the current collectors is 0 but has been specified above for demonstration purposes. With this option the Ohmic losses in the current collectors are assumed to be negligible meaning that the entire current collector is held at the same potential. This may be a good approximation for smaller pouch cells or even larger ones at low C-rates. However, for cells with larger aspect ratios the Ohmic losses in the current collectors become more important and regions of the cell may discharge or charge at different rates depending on their proximity to a tab. 

In [7]:
model_1D = pybamm.lithium_ion.SPMe({"dimensionality": 1,
                                    "current collector": "potential pair"})
sim_1D = pybamm.Simulation(model_1D, parameter_values=param)
sim_1D.solve([0, 3600])
sim_1D.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

Now we can observe the effects of dimensionality on a thermal model

In [8]:
model_1D_lumped = pybamm.lithium_ion.SPMe({"dimensionality": 1,
                                           "current collector": "potential pair",
                                           "thermal": "lumped"})
sim_1D_lumped = pybamm.Simulation(model_1D_lumped, parameter_values=param)
sim_1D_lumped.solve([0, 3600])

In [9]:
temp = sim_1D_lumped.solution["Cell temperature [K]"].entries
t_min = temp.min()
t_max = temp.max()
variable_limits = {"X-averaged cell temperature [K]": (t_min, t_max),
                   "Cell temperature [K]": (t_min, t_max),
                   "Volume-averaged cell temperature [K]": (t_min, t_max)}

In [10]:
sim_1D_lumped.plot([
    #"Terminal voltage [V]",
                    "X-averaged cell temperature [K]",
                    "Cell temperature [K]",
                    "Volume-averaged cell temperature [K]"], variable_limits=variable_limits)

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In [11]:
model_1D_x_lumped = pybamm.lithium_ion.SPMe({"dimensionality": 1,
                                             "current collector": "potential pair",
                                             "thermal": "x-lumped"})
sim_1D_x_lumped = pybamm.Simulation(model_1D_x_lumped, parameter_values=param)
sim_1D_x_lumped.solve([0, 3600])

In [12]:
temp = sim_1D_x_lumped.solution["Cell temperature [K]"].entries
t_min = temp.min()
t_max = temp.max()
variable_limits = {"X-averaged cell temperature [K]": (t_min, t_max),
                   "Cell temperature [K]": (t_min, t_max),
                   "Volume-averaged cell temperature [K]": (t_min, t_max)}
print(t_min, t_max)

298.15 300.16053281090683


In [13]:
temp.shape

(60, 10, 100)

In [14]:
sim_1D_x_lumped.plot(["X-averaged cell temperature [K]",
                      "Cell temperature [K]",
                      "Volume-averaged cell temperature [K]"],
                     #variable_limits=variable_limits,
                     variable_limits='tight',
                    )

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In [15]:
model_2D = pybamm.lithium_ion.SPMe({"dimensionality": 2,
                                    "current collector": "potential pair"})
sim_2D = pybamm.Simulation(model_2D, parameter_values=param)
sim_2D.solve([0, 3600])

In [16]:
sim_2D.plot(["Negative current collector potential [V]",
             "Positive current collector potential [V]"], variable_limits="tight")

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In [17]:
sim_2D.plot(["Current collector current density [A.m-2]",
            "Local voltage [V]"], variable_limits="tight")

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In [18]:
param.search('thickness')

Initial inner SEI thickness [m]	2.5e-09
Initial outer SEI thickness [m]	2.5e-09
Negative current collector thickness [m]	2.5e-05
Negative electrode thickness [m]	0.0001
Positive current collector thickness [m]	2.5e-05
Positive electrode thickness [m]	0.0001
Separator thickness [m]	2.5e-05


In [19]:
param.search('conductivity')

Electrolyte conductivity [S.m-1]	<function electrolyte_conductivity_Capiglia1999 at 0x000001EB37203F70>
Inner SEI electron conductivity [S.m-1]	8.95e-14
Negative current collector conductivity [S.m-1]	59600000.0
Negative current collector thermal conductivity [W.m-1.K-1]	401.0
Negative electrode conductivity [S.m-1]	100.0
Negative electrode thermal conductivity [W.m-1.K-1]	1.7
Positive current collector conductivity [S.m-1]	35500000.0
Positive current collector thermal conductivity [W.m-1.K-1]	237.0
Positive electrode conductivity [S.m-1]	10.0
Positive electrode thermal conductivity [W.m-1.K-1]	2.1
Separator thermal conductivity [W.m-1.K-1]	0.16


In [20]:
param.search('transfer coefficient')

Edge heat transfer coefficient [W.m-2.K-1]	1.0
Negative current collector surface heat transfer coefficient [W.m-2.K-1]	1.0
Negative electrode charge transfer coefficient	0.5
Negative tab heat transfer coefficient [W.m-2.K-1]	1.0
Positive current collector surface heat transfer coefficient [W.m-2.K-1]	1.0
Positive electrode charge transfer coefficient	0.5
Positive tab heat transfer coefficient [W.m-2.K-1]	1.0
Total heat transfer coefficient [W.m-2.K-1]	1.0


In [21]:
sim_2D.plot(["Average negative particle concentration [mol.m-3]",
             "Average positive particle concentration [mol.m-3]"], variable_limits="tight")


interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…